In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
openai.api_key

'voc-1570125386112272321848266d1e0a2cc0324.85981571'

In [3]:
def load_local_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.readlines()
    return data

In [4]:
def get_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response['data'][0]['embedding']

In [5]:
def generate_local_embeddings(local_data):
    return np.array([get_embedding(text) for text in local_data])

In [6]:
def get_local_response(question, local_data, local_embeddings):
    question_embedding = np.array(get_embedding(question)).reshape(1, -1)
    cosine_similarities = cosine_similarity(question_embedding, local_embeddings).flatten()
    most_similar_index = cosine_similarities.argmax()
    return local_data[most_similar_index]

In [7]:
def get_openai_response(question):
    response = openai.Completion.create(
        #engine="text-davinci-003",  # or another model
        engine="gpt-3.5-turbo-instruct",
        prompt=question,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].text.strip()

In [8]:
def get_combined_response(question, local_data, local_embeddings):
    local_response = get_local_response(question, local_data, local_embeddings)
    openai_response = get_openai_response(question)
    
    combined_response = f"Local Data: {local_response}\nOpenAI: {openai_response}"
    return combined_response

In [9]:
local_data = load_local_data('demo.txt')
import numpy as np

In [10]:
local_embeddings = generate_local_embeddings(local_data)

In [11]:
question = "What is the capital of India?"

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
response = get_combined_response(question, local_data, local_embeddings)

In [13]:
print(response)

Local Data: What is the capital of France? The capital of France is India.

OpenAI: The capital of India is New Delhi.
